This is the jupyter notebook pertaining to the GLEIF_Backfill.py file.

This code is responsible for obtaining all Level 1 and Level 2 Relationship data from the GLIEF, and backfilling the data and adding whatever features are needed to make the data accessible for further use

In [6]:
from bs4 import BeautifulSoup
import os
import requests
import logging
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import sqlite3
import zipfile
import pandas as pd
import json
import pickle
import bigjson
import json
import sys
current_directory = os.getcwd()
display(current_directory)
target_directory = os.path.abspath(os.path.join(current_directory))
display(target_directory)
sys.path.append(target_directory)
from Infastructure import System_Helpers

'c:\\Users\\mattp\\Work_Related\\Gradient_Trading\\GLEIF_Pipeline\\GLEIF_Pipeline\\Notebooks\\Baselines'

'c:\\Users\\mattp\\Work_Related\\Gradient_Trading\\GLEIF_Pipeline\\GLEIF_Pipeline\\Notebooks\\Baselines'

ModuleNotFoundError: No module named 'Infastructure'

In [2]:
from bs4 import BeautifulSoup
import os
import requests
import logging
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import sqlite3
import zipfile
import pandas as pd
import json
import pickle
import bigjson
import json
import sys
current_directory = os.getcwd()
target_directory = os.path.abspath(os.path.join(current_directory, "..", "..", ".."))
sys.path.append(target_directory)
from D_Infastructure import System_Helpers

class GLEIF_Backill_Helpers:
    def __init__(self, bool_Level_1 = False, bool_Level_2_Trees = False, bool_Level_2_Reporting_Exceptions = False):
        self.bool_Level_1 = bool_Level_1
        self.bool_Level_2_Trees = bool_Level_2_Trees
        self.bool_Level_2_Reporting_Exceptions = bool_Level_2_Reporting_Exceptions
        

    def get_level_download_links(self):
        """
        This function uses selenium to webscrape the download link for all Level 1 Data in the GLEIF database.
        
        @return: str_download_link - the link which is used to download the entire GLEIF level 1
        """

        driver_path = (r"C:\Drivers\Google\chromedriver-win64\chromedriver-win64\chromedriver.exe")
        service = Service(driver_path)
        driver = webdriver.Chrome(service=service)
        driver.get(url = "https://www.gleif.org/en/lei-data/gleif-golden-copy/download-the-golden-copy#/")

        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'CybotCookiebotDialogBodyButton'))
        )

        cookie_button.click()

        download_buttons = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'gc-download-button'))
        )
        
        if self.bool_Level_1 == True:
            download_buttons[0].click()
        if self.bool_Level_2_Trees == True:
            download_buttons[1].click()
        if self.bool_Level_2_Reporting_Exceptions == True:
            download_buttons[2].click()
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        driver.close()

        str_download_link = ((soup.find_all("a" , class_ = "gc-icon gc-icon--json"))[0])["href"]
        
        return str_download_link        
    
    def create_sql_instance(self, str_db_name):
        # Connect to the SQLite database
        conn = sqlite3.connect(f'{str_db_name}.db', timeout=10)
        # Create the schema
        self.create_database_schema(conn)
        return conn
        
    def create_tables(self, conn):
        cursor = conn.cursor()
        cursor.execute("PRAGMA foreign_keys = ON;")

        schema = """
        -- Table: entity
        CREATE TABLE IF NOT EXISTS entity (
            lei TEXT PRIMARY KEY,
            legal_name TEXT NOT NULL,
            legal_jurisdiction TEXT,
            entity_category TEXT,
            entity_status TEXT,
            entity_creation_date TEXT,
            successor_lei TEXT,
            successor_entity_name TEXT,
            entity_legal_form_code TEXT,
            other_legal_form TEXT,
            FOREIGN KEY (successor_lei) REFERENCES entity(lei)
        );

        -- Table: address
        CREATE TABLE IF NOT EXISTS address (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            entity_lei TEXT NOT NULL,
            address_type TEXT NOT NULL,
            first_address_line TEXT,
            additional_address_line TEXT,
            city TEXT,
            country TEXT,
            postal_code TEXT,
            FOREIGN KEY (entity_lei) REFERENCES entity(lei)
        );

        -- Table: registration_authority
        CREATE TABLE IF NOT EXISTS registration_authority (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            entity_lei TEXT NOT NULL,
            registration_authority_id TEXT,
            registration_authority_entity_id TEXT,
            FOREIGN KEY (entity_lei) REFERENCES entity(lei)
        );

        -- Table: registration
        CREATE TABLE IF NOT EXISTS registration (
            entity_lei TEXT PRIMARY KEY,
            initial_registration_date TEXT,
            last_update_date TEXT,
            registration_status TEXT,
            next_renewal_date TEXT,
            managing_lou TEXT,
            validation_sources TEXT,
            FOREIGN KEY (entity_lei) REFERENCES entity(lei)
        );

        -- Table: validation_authority
        CREATE TABLE IF NOT EXISTS validation_authority (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            entity_lei TEXT NOT NULL,
            is_other_authority INTEGER NOT NULL DEFAULT 0,
            validation_authority_id TEXT,
            other_validation_authority_id TEXT,
            validation_authority_entity_id TEXT,
            FOREIGN KEY (entity_lei) REFERENCES entity(lei)
        );

        -- Table: legal_entity_events
        CREATE TABLE IF NOT EXISTS legal_entity_events (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            entity_lei TEXT NOT NULL,
            legal_entity_event_type TEXT,
            legal_entity_event_effective_date TEXT,
            legal_entity_event_recorded_date TEXT,
            validation_documents TEXT,
            validation_reference TEXT,
            affected_fields TEXT,
            group_type TEXT,
            event_status TEXT,
            group_id TEXT,
            group_sequence_no INTEGER,
            FOREIGN KEY (entity_lei) REFERENCES entity(lei)
        );

        -- Table: geocoding
        CREATE TABLE IF NOT EXISTS geocoding (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            entity_lei TEXT NOT NULL,
            original_address TEXT,
            geocoding_failed TEXT,
            relevance REAL,
            lat REAL,
            lng REAL,
            geocoding_date TEXT,
            bounding_box TEXT,
            match_level TEXT,
            formatted_address TEXT,
            mapped_location_id TEXT,
            mapped_city TEXT,
            mapped_district TEXT,
            mapped_country TEXT,
            FOREIGN KEY (entity_lei) REFERENCES entity(lei)
        );

        -- Table: conformity
        CREATE TABLE IF NOT EXISTS conformity (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            entity_lei TEXT NOT NULL,
            conformity_flag TEXT,
            FOREIGN KEY (entity_lei) REFERENCES entity(lei)
        );

        -- Indexes for Performance Optimization
        CREATE INDEX IF NOT EXISTS idx_address_entity_lei ON address(entity_lei);
        CREATE INDEX IF NOT EXISTS idx_registration_authority_entity_lei ON registration_authority(entity_lei);
        CREATE INDEX IF NOT EXISTS idx_validation_authority_entity_lei ON validation_authority(entity_lei);
        CREATE INDEX IF NOT EXISTS idx_legal_entity_events_entity_lei ON legal_entity_events(entity_lei);
        CREATE INDEX IF NOT EXISTS idx_geocoding_entity_lei ON geocoding(entity_lei);
        CREATE INDEX IF NOT EXISTS idx_conformity_entity_lei ON conformity(entity_lei);
        """
        cursor.executescript(schema)
        conn.commit()
    
    def unpacking_GLEIF_zip_files(self , str_download_link , str_zip_file_path_name , str_unpacked_zip_file_name):
        session = requests.Session()
        zip_file = session.get(url = str_download_link)

        with open(rf"file_lib\{str_zip_file_path_name}", 'wb') as f:
            f.write(zip_file.content)

        with zipfile.ZipFile(rf"file_lib\{str_zip_file_path_name}", 'r') as zip_ref:
            os.makedirs(rf"file_lib\{str_unpacked_zip_file_name}", exist_ok=True)
            zip_ref.extractall(rf"file_lib\{str_unpacked_zip_file_name}")
        
        str_unpacked_zip_file_name = os.listdir(rf"file_lib\{str_unpacked_zip_file_name}")[0]
        str_json_file_path = rf"file_lib\{str_unpacked_zip_file_name}" + "\\" + str_unpacked_zip_file_name
        
        return str_json_file_path
    
    
    
    

ModuleNotFoundError: No module named 'D_Infastructure'

In [4]:
if not os.path.exists("file_lib"):
    os.makedirs("../file_lib")

In [ ]:
class GLEIFLevel1Data:
    def __init__(self , bool_log = True , db_name = "GLEIF_Data.db" , bool_downloaded = True):
        if bool_log:
            if not os.path.exists("logging"): #if the logging folder doesnt exist in the directory, make it
                os.makedirs("../logging")
            logging.basicConfig(filename = "logging/Company_Facts_Baseline.log" , level = logging.DEBUG, format = '%(levelname)s: %(message)s' , filemode = "w")

        if not bool_downloaded:
            if not os.path.exists("file_lib"):
                os.makedirs("../file_lib")
                
        obj_backfill_helpers = GLEIF_Backill_Helpers(bool_Level_1 = True)
        str_level_1_download_link = obj_backfill_helpers.get_level_download_links()
        self.str_json_file_path = obj_backfill_helpers.unpacking_GLEIF_zip_files(str_download_link = str_level_1_download_link)

    def helper_parse_entity_data(self):
    
    
    def parse_lei_record(self, record):
        # Extract data for the entity table
        entity_data = {
            'lei': record.get('LEI', {}).get('$'),
            'legal_name': record.get('Entity', {}).get('LegalName', {}).get('$'),
            'legal_jurisdiction': record.get('Entity', {}).get('LegalJurisdiction', {}).get('$'),
            'entity_category': record.get('Entity', {}).get('EntityCategory', {}).get('$'),
            'entity_status': record.get('Entity', {}).get('EntityStatus', {}).get('$'),
            'entity_creation_date': record.get('Entity', {}).get('EntityCreationDate', {}).get('$'),
            'successor_lei': record.get('Entity', {}).get('SuccessorEntity', {}).get('SuccessorLEI', {}).get('$'),
            'successor_entity_name': record.get('Entity', {}).get('SuccessorEntity', {}).get('SuccessorName', {}).get('$'),
            'entity_legal_form_code': record.get('Entity', {}).get('LegalForm', {}).get('EntityLegalFormCode', {}).get('$'),
            'other_legal_form': record.get('Entity', {}).get('LegalForm', {}).get('OtherLegalForm', {}).get('$')
        }

        # Extract data for the address table
        addresses = []
        for address_type in ['LegalAddress', 'HeadquartersAddress']:
            address_info = record.get('Entity', {}).get(address_type)
            if address_info:
                additional_address_line = address_info.get('AdditionalAddressLine')
                # Handle case where AdditionalAddressLine is a list
                if isinstance(additional_address_line, list):
                    additional_address_line = ', '.join([
                        line.get('$', '') if isinstance(line, dict) else line
                        for line in additional_address_line
                    ])
                elif isinstance(additional_address_line, dict):
                    additional_address_line = additional_address_line.get('$', '')
                else:
                    additional_address_line = additional_address_line

                address_data = {
                    'entity_lei': entity_data['lei'],
                    'address_type': 'LEGAL' if address_type == 'LegalAddress' else 'HEADQUARTERS',
                    'first_address_line': address_info.get('FirstAddressLine', {}).get('$'),
                    'additional_address_line': additional_address_line,
                    'city': address_info.get('City', {}).get('$'),
                    'country': address_info.get('Country', {}).get('$'),
                    'postal_code': address_info.get('PostalCode', {}).get('$')
                }
                addresses.append(address_data)

        # Extract data for the registration_authority table
        registration_authority_info = record.get('Entity', {}).get('RegistrationAuthority')
        registration_authority_data = None
        if registration_authority_info:
            registration_authority_data = {
                'entity_lei': entity_data['lei'],
                'registration_authority_id': registration_authority_info.get('RegistrationAuthorityID', {}).get('$'),
                'registration_authority_entity_id': registration_authority_info.get('RegistrationAuthorityEntityID', {}).get('$')
            }

        # Extract data for the registration table
        registration_info = record.get('Registration', {})
        registration_data = {
            'entity_lei': entity_data['lei'],
            'initial_registration_date': registration_info.get('InitialRegistrationDate', {}).get('$'),
            'last_update_date': registration_info.get('LastUpdateDate', {}).get('$'),
            'registration_status': registration_info.get('RegistrationStatus', {}).get('$'),
            'next_renewal_date': registration_info.get('NextRenewalDate', {}).get('$'),
            'managing_lou': registration_info.get('ManagingLOU', {}).get('$'),
            'validation_sources': registration_info.get('ValidationSources', {}).get('$')
        }

        # Extract data for the validation_authority table
        validation_authority_info = registration_info.get('ValidationAuthority')
        validation_authorities = []
        if validation_authority_info:
            validation_authority_data = {
                'entity_lei': entity_data['lei'],
                'is_other_authority': 0,
                'validation_authority_id': validation_authority_info.get('ValidationAuthorityID', {}).get('$'),
                'other_validation_authority_id': None,
                'validation_authority_entity_id': validation_authority_info.get('ValidationAuthorityEntityID', {}).get('$')
            }
            validation_authorities.append(validation_authority_data)

        return {
            'entity': entity_data,
            'addresses': addresses,
            'registration_authority': registration_authority_data,
            'registration': registration_data,
            'validation_authorities': validation_authorities
            # Include other parsed data as needed
        }

    def insert_entity(self, conn, entity_data):
        cursor = conn.cursor()
        sql = """
        INSERT OR REPLACE INTO entity (
            lei,
            legal_name,
            legal_jurisdiction,
            entity_category,
            entity_status,
            entity_creation_date,
            successor_lei,
            successor_entity_name,
            entity_legal_form_code,
            other_legal_form
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        """
        cursor.execute(sql, (
            entity_data['lei'],
            entity_data['legal_name'],
            entity_data.get('legal_jurisdiction'),
            entity_data.get('entity_category'),
            entity_data.get('entity_status'),
            entity_data.get('entity_creation_date'),
            entity_data.get('successor_lei'),
            entity_data.get('successor_entity_name'),
            entity_data.get('entity_legal_form_code'),
            entity_data.get('other_legal_form')
        ))
        # Do not commit here; commit in the main function to optimize performance

    def insert_address(self, conn, address_data):
        cursor = conn.cursor()
        sql = """
        INSERT INTO address (
            entity_lei,
            address_type,
            first_address_line,
            additional_address_line,
            city,
            country,
            postal_code
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """
        cursor.execute(sql, (
            address_data['entity_lei'],
            address_data['address_type'],
            address_data.get('first_address_line'),
            address_data.get('additional_address_line'),
            address_data.get('city'),
            address_data.get('country'),
            address_data.get('postal_code')
        ))
        # Do not commit here; commit in the main function to optimize performance

    def insert_registration_authority(self, conn, registration_authority_data):
        cursor = conn.cursor()
        sql = """
        INSERT INTO registration_authority (
            entity_lei,
            registration_authority_id,
            registration_authority_entity_id
        ) VALUES (?, ?, ?);
        """
        cursor.execute(sql, (
            registration_authority_data['entity_lei'],
            registration_authority_data.get('registration_authority_id'),
            registration_authority_data.get('registration_authority_entity_id')
        ))
        # Do not commit here; commit in the main function to optimize performance

    def insert_registration(self, conn, registration_data):
        cursor = conn.cursor()
        sql = """
        INSERT OR REPLACE INTO registration (
            entity_lei,
            initial_registration_date,
            last_update_date,
            registration_status,
            next_renewal_date,
            managing_lou,
            validation_sources
        ) VALUES (?, ?, ?, ?, ?, ?, ?);
        """
        cursor.execute(sql, (
            registration_data['entity_lei'],
            registration_data.get('initial_registration_date'),
            registration_data.get('last_update_date'),
            registration_data.get('registration_status'),
            registration_data.get('next_renewal_date'),
            registration_data.get('managing_lou'),
            registration_data.get('validation_sources')
        ))
        # Do not commit here; commit in the main function to optimize performance

    def insert_validation_authority(self, conn, validation_authority_data):
        cursor = conn.cursor()
        sql = """
        INSERT INTO validation_authority (
            entity_lei,
            is_other_authority,
            validation_authority_id,
            other_validation_authority_id,
            validation_authority_entity_id
        ) VALUES (?, ?, ?, ?, ?);
        """
        cursor.execute(sql, (
            validation_authority_data['entity_lei'],
            validation_authority_data.get('is_other_authority', 0),
            validation_authority_data.get('validation_authority_id'),
            validation_authority_data.get('other_validation_authority_id'),
            validation_authority_data.get('validation_authority_entity_id')
        ))
        # Do not commit here; commit in the main function to optimize performance

    def storing_GLEIF_data_in_database(self):
        # Download and unpack the GLEIF data
        str_level_1_download_link = self.obj_backfill_helpers.get_level_download_links()
        str_json_file_path = self.obj_backfill_helpers.unpacking_GLEIF_zip_files(str_download_link = str_level_1_download_link, str_zip_file_path=self.str_level_1_zip_file_path, str_unpacked_zip_file_path = self.str_level_1_unpacked_zip_file_path)
        # Create database and schema
        conn = self.obj_backfill_helpers.create_sql_instance(str_db_name="GLEIF_Data")

        with open(str_json_file_path, 'rb') as file:
            dict_leis = bigjson.load(file)
            # Begin a transaction
            conn.execute('BEGIN')
            try:
                for dict_lei in dict_leis['LEIRecords']:
                    record = dict_lei.to_python()
                    parsed_data = self.parse_lei_record(record)
                    # Insert data into tables
                    self.insert_entity(conn, parsed_data['entity'])
                    for address in parsed_data['addresses']:
                        self.insert_address(conn, address)
                    self.insert_registration_authority(conn, parsed_data['registration_authority'])
                    self.insert_registration(conn, parsed_data['registration'])
                    for validation_authority in parsed_data['validation_authorities']:
                        self.insert_validation_authority(conn, validation_authority)
                    # Handle other tables similarly
                conn.commit()
            except Exception as e:
                print(f"An error occurred: {e}")
                conn.rollback()
            finally:
                conn.close()
        
        obj_system_helpers = System_Helpers.SystemHelpers()
        obj_system_helpers.delete_file_directory(str_file_path = self.str_level_1_unpacked_zip_file_path , bool_directory = True)
        obj_system_helpers.delete_file_directory(str_file_path = self.str_level_1_zip_file_path , bool_file = True)
                    

In [1]:
class GLEIFLevel2Data:
    def __init__(self):
        self.str_level_2_unpacked_zip_file_path = r"C:\Users\mattp\Work_Related\Systematic_Trading\Library\Zip_Files\GLEIF\Level_2_Data\RR_CDF_Data\Unpacked_Zip"
        self.str_level_2_zip_file_path = r"C:\Users\mattp\Work_Related\Systematic_Trading\Library\Zip_Files\GLEIF\Level_2_Data\RR_CDF_Data\Level_2_RR_CDF.zip"
        self.obj_backfill_helpers = GLEIF_Backill_Helpers(bool_Level_2_Trees = True)

    def insert_json_data(self, json_data , conn , cursor , str_table_name):
        cursor.execute(f'''
        INSERT INTO {str_table_name}  (data)
        VALUES (?)
        ''', (json.dumps(json_data),))
        conn.commit()
    
    def storing_GLEIF_data_in_database(self):
        str_level_2_download_link = self.obj_data_helpers.get_level_download_links()
        str_json_file_path = self.obj_data_helpers.unpacking_GLEIF_zip_files(str_download_link = str_level_2_download_link , str_zip_file_path = self.str_level_2_zip_file_path , str_unpacked_zip_file_path = self.str_level_2_unpacked_zip_file_path)
        conn, cursor = self.obj_data_helpers.create_sql_instance(str_table_name = "Level_2_Tree_Data" , str_db_name = "GLEIF_Data")
        
        with open(str_json_file_path, 'r' , encoding='utf-8') as file:
            test = bigjson.load(file)
            for dict_lei in test["relations"]:
                self.insert_json_data(json_data = dict_lei.to_python() , conn = conn , cursor = cursor , str_table_name = "Level_2_Tree_Data")
                    
            
        conn.close()

        obj_system_helpers = System_Helpers.SystemHelpers()
        obj_system_helpers.delete_file_directory(str_file_path = self.str_level_2_unpacked_zip_file_path , bool_directory = True)
        obj_system_helpers.delete_file_directory(str_file_path = self.str_level_2_zip_file_path , bool_file = True)

In [6]:
import sqlite3

In [11]:
def rename_table(old_table_name, new_table_name):
    """
    Renames a table in the SQLite database.

    Parameters:
        db_name (str): The SQLite database name.
        old_table_name (str): The current name of the table.
        new_table_name (str): The new name for the table.
    """
    try:
        # Connect to the database
        conn = sqlite3.connect(r"C:\Users\mattp\Work_Related\Systematic_Trading\Library\B_Notebooks\GLIEF_company_data_pipeline\GLEIF_Data.db", check_same_thread=False)
        cursor = conn.cursor()

        # Rename the table
        cursor.execute(f"ALTER TABLE {old_table_name} RENAME TO {new_table_name}")
        conn.commit()

        print(f"Table '{old_table_name}' has been renamed to '{new_table_name}'.")
    except sqlite3.Error as e:
        print(f"Error renaming table: {e}")
    finally:
        conn.close()